### Predictive modeling of Orders (Order_payment, review, items and Orders)


#### Juntando todas as tabelas em uma só

In [1]:
import pandas as pd
import snowflake.connector
import os
from dotenv import load_dotenv
from pathlib import Path

# Carrega as variáveis de ambiente
env_path = Path('.') / 'environment.env'
load_dotenv(dotenv_path=env_path)
SF_USER = os.getenv("SF_USER")
SF_PASSWORD = os.getenv("SF_PASSWORD")
SF_ACCOUNT = os.getenv("SF_ACCOUNT")
SF_WAREHOUSE = os.getenv("SF_WAREHOUSE")
SF_DATABASE = os.getenv("SF_DATABASE")
SF_SCHEMA = os.getenv("SF_SCHEMA")

# Conecta ao Snowflake
conn = snowflake.connector.connect(
    user=SF_USER,
    password=SF_PASSWORD,
    account=SF_ACCOUNT,
    warehouse=SF_WAREHOUSE,
    database=SF_DATABASE,
    schema=SF_SCHEMA
)

# Carrega as tabelas
print("Carregando tabelas do Snowflake...")
df_orders = pd.read_sql("SELECT * FROM orders_refined", conn)
df_orders_reviews = pd.read_sql("SELECT * FROM order_reviews_refined", conn)
df_order_payments = pd.read_sql("SELECT * FROM order_payments_refined", conn)
df_order_items = pd.read_sql("SELECT * FROM order_items_refined", conn)
df_products = pd.read_sql("SELECT * FROM products_refined", conn)
# 💡 NOVO: Carregando a tabela de clientes
df_customers = pd.read_sql("SELECT * FROM customers_refined", conn)

# Fecha a conexão
conn.close()

# Padroniza os nomes das colunas
for df in [df_orders, df_orders_reviews, df_order_payments, df_order_items, df_products, df_customers]:
    df.columns = df.columns.str.lower()

# Realiza as junções sequenciais
print("Unindo as tabelas...")
df_full_orders = df_orders.merge(df_orders_reviews, on='order_id', how='left')
df_full_orders = df_full_orders.merge(df_order_payments, on='order_id', how='left')
df_full_orders = df_full_orders.merge(df_order_items, on='order_id', how='left')
df_full_orders = df_full_orders.merge(df_products, on='product_id', how='left')
# 💡 NOVO: Juntando a tabela de clientes
df_full_orders = df_full_orders.merge(
    df_customers[['customer_id', 'customer_state', 'customer_zip_code_prefix']],
    on='customer_id',
    how='left'
)

print("Junção completa. O DataFrame final tem o formato:", df_full_orders.shape)
print(df_full_orders.head())

Carregando tabelas do Snowflake...


C:\Users\pacie\AppData\Local\Temp\ipykernel_28908\3132649183.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_orders = pd.read_sql("SELECT * FROM orders_refined", conn)
C:\Users\pacie\AppData\Local\Temp\ipykernel_28908\3132649183.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_orders_reviews = pd.read_sql("SELECT * FROM order_reviews_refined", conn)
C:\Users\pacie\AppData\Local\Temp\ipykernel_28908\3132649183.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_order_payments = pd.read_sql("SELECT * FROM o

Unindo as tabelas...
Junção completa. O DataFrame final tem o formato: (476572, 56)
                           order_id                       customer_id  \
0  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
1  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
2  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4  85ce859fd6dc634de8d2f1e290444043  059f7fc5719c7da6cbafe370971a8d70   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      1511033286000000000  1511034359000000000   
1    delivered      1511033286000000000  1511034359000000000   
2    delivered      1511033286000000000  1511034359000000000   
3    delivered      1511033286000000000  1511034359000000000   
4    delivered      1511222621000000000  1511223262000000000   

  order_delivered_carrier_date order_delivered_customer_date  \
0          15113579990000000

### Salvando de volta para csv (com todas as tabelas) para usar exemplos reais

In [38]:
import pandas as pd
import snowflake.connector
import os
from dotenv import load_dotenv
from pathlib import Path

# Carrega as variáveis de ambiente
env_path = Path('.') / 'environment.env'
load_dotenv(dotenv_path=env_path)
SF_USER = os.getenv("SF_USER")
SF_PASSWORD = os.getenv("SF_PASSWORD")
SF_ACCOUNT = os.getenv("SF_ACCOUNT")
SF_WAREHOUSE = os.getenv("SF_WAREHOUSE")
SF_DATABASE = os.getenv("SF_DATABASE")
SF_SCHEMA = os.getenv("SF_SCHEMA")

# Conecta ao Snowflake
conn = snowflake.connector.connect(
    user=SF_USER,
    password=SF_PASSWORD,
    account=SF_ACCOUNT,
    warehouse=SF_WAREHOUSE,
    database=SF_DATABASE,
    schema=SF_SCHEMA
)

# Carrega as tabelas
print("Carregando tabelas do Snowflake...")
df_orders = pd.read_sql("SELECT * FROM orders_refined", conn)
df_orders_reviews = pd.read_sql("SELECT * FROM order_reviews_refined", conn)
df_order_payments = pd.read_sql("SELECT * FROM order_payments_refined", conn)
df_order_items = pd.read_sql("SELECT * FROM order_items_refined", conn)
df_products = pd.read_sql("SELECT * FROM products_refined", conn)
df_customers = pd.read_sql("SELECT * FROM customers_refined", conn)

# Fecha a conexão
conn.close()

# Padroniza os nomes das colunas
for df in [df_orders, df_orders_reviews, df_order_payments, df_order_items, df_products, df_customers]:
    df.columns = df.columns.str.lower()

# Realiza as junções sequenciais
print("Unindo as tabelas...")
df_full_orders = df_orders.merge(df_orders_reviews, on='order_id', how='left')
df_full_orders = df_full_orders.merge(df_order_payments, on='order_id', how='left')
df_full_orders = df_full_orders.merge(df_order_items, on='order_id', how='left')
df_full_orders = df_full_orders.merge(df_products, on='product_id', how='left')
df_full_orders = df_full_orders.merge(
    df_customers[['customer_id', 'customer_state', 'customer_zip_code_prefix']],
    on='customer_id',
    how='left'
)

# Salva o DataFrame completo em um arquivo CSV
print("Salvando o DataFrame completo em olist_full_dataset.csv...")
df_full_orders.to_csv('olist_full_dataset.csv', index=False)

print("Processo concluído. O arquivo 'olist_full_dataset.csv' foi criado com sucesso!")

Carregando tabelas do Snowflake...


C:\Users\pacie\AppData\Local\Temp\ipykernel_26324\1455172169.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_orders = pd.read_sql("SELECT * FROM orders_refined", conn)
C:\Users\pacie\AppData\Local\Temp\ipykernel_26324\1455172169.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_orders_reviews = pd.read_sql("SELECT * FROM order_reviews_refined", conn)
C:\Users\pacie\AppData\Local\Temp\ipykernel_26324\1455172169.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_order_payments = pd.read_sql("SELECT * FROM o

Unindo as tabelas...
Salvando o DataFrame completo em olist_full_dataset.csv...
Processo concluído. O arquivo 'olist_full_dataset.csv' foi criado com sucesso!


### Criando dataset monthly_revenue para predição


In [4]:
import pandas as pd

# Supondo que df_orders e df_order_payments já estão carregados

# 💡 FUNÇÃO DE CONVERSÃO CORRIGIDA
# Esta função garante que timestamps muito grandes sejam convertidos corretamente
def safe_timestamp_to_datetime(series):
    series = pd.to_numeric(series, errors='coerce')
    if series.dropna().empty:
        return pd.NaT
    max_val = series.max()
    if max_val > 1e18:           # nanosegundos
        unit = 'ns'
    elif max_val > 1e12:         # milissegundos
        unit = 'ms'
    else:                        # segundos
        unit = 's'
    return pd.to_datetime(series, unit=unit, errors='coerce')


# --- 1. Junção e Preparação dos Dados ---
print("Preparando dados para a série temporal de faturamento...")

# Junta pedidos e pagamentos
df_revenue = df_orders.merge(
    df_order_payments[['order_id', 'payment_value']],
    on='order_id',
    how='inner'
)

# 💡 CORREÇÃO: Usa a função segura para converter o timestamp
df_revenue['order_purchase_datetime'] = safe_timestamp_to_datetime(df_revenue['order_purchase_timestamp'])


# --- 2. Criação da Série Temporal de Faturamento Mensal ---
print("Criando a série temporal de faturamento mensal...")

# Agrupa por mês e soma o valor dos pagamentos
monthly_revenue = df_revenue.groupby(
    pd.Grouper(key='order_purchase_datetime', freq='M')
)['payment_value'].sum().reset_index()

# Renomeia as colunas
monthly_revenue.rename(columns={'order_purchase_datetime': 'data', 'payment_value': 'faturamento_mensal'}, inplace=True)


# --- 3. Criação das Features de Lag e Temporais ---
print("Criando features de lag e temporais...")

# Features de Lag (faturamento dos meses anteriores)
monthly_revenue['faturamento_mes_anterior'] = monthly_revenue['faturamento_mensal'].shift(1)
monthly_revenue['faturamento_2_meses_atras'] = monthly_revenue['faturamento_mensal'].shift(2)
monthly_revenue['faturamento_3_meses_atras'] = monthly_revenue['faturamento_mensal'].shift(3)

# Features Temporais
monthly_revenue['mes'] = monthly_revenue['data'].dt.month
monthly_revenue['ano'] = monthly_revenue['data'].dt.year

# Remove as linhas com valores nulos (os primeiros meses que não têm lag)
monthly_revenue.dropna(inplace=True)

# Exibe o DataFrame final
print("\nDataFrame de Faturamento Mensal com Features de Lag:")
print(monthly_revenue.head())
print(f"\nDataFrame final tem o formato: {monthly_revenue.shape}")
# --- Adicione este trecho de código ao final da sua célula ---

# Salva o DataFrame final em um arquivo CSV
print("Salvando o DataFrame de faturamento mensal em 'olist_monthly_revenue.csv'...")
monthly_revenue.to_csv('olist_monthly_revenue.csv', index=False)

print("Arquivo 'olist_full_dataset.csv' criado com sucesso!")

Preparando dados para a série temporal de faturamento...
Criando a série temporal de faturamento mensal...


C:\Users\pacie\AppData\Local\Temp\ipykernel_28908\1667327117.py:40: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  pd.Grouper(key='order_purchase_datetime', freq='M')


Criando features de lag e temporais...

DataFrame de Faturamento Mensal com Features de Lag:
        data                                 faturamento_mensal  \
3 2016-12-31                                              19.62   
4 2017-01-31  19.6219.6211.6216.6218.6219.6220.6219.6219.621...   
5 2017-02-28  174.5452.7847.6870.28176.3970.2840.52179.3559....   
6 2017-03-31  52.1849.96118.0343.05188.1849.52147.0666.9940....   
7 2017-04-30  84.1536.86204.08161.05103.1143.05114.79113.768...   

                            faturamento_mes_anterior  \
3                                                  0   
4                                              19.62   
5  19.6219.6211.6216.6218.6219.6220.6219.6219.621...   
6  174.5452.7847.6870.28176.3970.2840.52179.3559....   
7  52.1849.96118.0343.05188.1849.52147.0666.9940....   

                           faturamento_2_meses_atras  \
3  109.3445.4639.0935.6153.73133.4640.95154.5792....   
4                                                  0   

### Treinando o modelo para prever o faturamento de um mês X

In [9]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import numpy as np

# Supondo que df_orders e df_order_payments já estão carregados na sua sessão

# 💡 FUNÇÃO DE CONVERSÃO: Necessária para lidar com os timestamps
def safe_timestamp_to_datetime(series):
    series = pd.to_numeric(series, errors='coerce')
    if series.dropna().empty:
        return pd.NaT
    max_val = series.max()
    if max_val > 1e18:           # nanosegundos
        unit = 'ns'
    elif max_val > 1e12:         # milissegundos
        unit = 'ms'
    else:                        # segundos
        unit = 's'
    return pd.to_datetime(series, unit=unit, errors='coerce')


# --- 1. Criação e Limpeza da Série Temporal de Faturamento ---
print("Recriando e limpando o DataFrame de faturamento mensal...")

df_revenue = df_orders.merge(
    df_order_payments[['order_id', 'payment_value']],
    on='order_id',
    how='inner'
)

df_revenue['payment_value'] = pd.to_numeric(df_revenue['payment_value'], errors='coerce')
df_revenue['order_purchase_datetime'] = safe_timestamp_to_datetime(df_revenue['order_purchase_timestamp'])

monthly_revenue = df_revenue.groupby(
    pd.Grouper(key='order_purchase_datetime', freq='M')
)['payment_value'].sum().reset_index()

monthly_revenue.rename(columns={'order_purchase_datetime': 'data', 'payment_value': 'faturamento_mensal'}, inplace=True)

# Criação das Features de Lag e Temporais
monthly_revenue['faturamento_mes_anterior'] = monthly_revenue['faturamento_mensal'].shift(1)
monthly_revenue['faturamento_2_meses_atras'] = monthly_revenue['faturamento_mensal'].shift(2)
monthly_revenue['faturamento_3_meses_atras'] = monthly_revenue['faturamento_mensal'].shift(3)

monthly_revenue['mes'] = monthly_revenue['data'].dt.month
monthly_revenue['ano'] = monthly_revenue['data'].dt.year

monthly_revenue.dropna(inplace=True)


# --- 2. Treinamento e Predição com Random Forest Regressor ---
print("\n--- Previsão de Faturamento do Último Mês ---")

# 💡 CORREÇÃO: Usando o Random Forest Regressor como modelo principal
features = [
    'faturamento_mes_anterior',
    'faturamento_2_meses_atras',
    'faturamento_3_meses_atras',
    'mes',
    'ano'
]

X = monthly_revenue[features]
y = monthly_revenue['faturamento_mensal']

X_train = X.iloc[:-1]
y_train = y.iloc[:-1]
X_test = X.iloc[-1:]
y_test = y.iloc[-1:]

print(f"Dados de treino: {len(X_train)} meses")
print(f"Dados de teste: {len(X_test)} mês")

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# --- 3. Análise e Comparação ---
print("\n--- Análise da Predição ---")
print(f"Faturamento real do último mês: R$ {y_test.values[0]:.2f}")
print(f"Faturamento previsto para o último mês: R$ {y_pred[0]:.2f}")

diferenca = y_test.values[0] - y_pred[0]
print(f"Diferença (Real - Previsto): R$ {diferenca:.2f}")

erro_percentual = (abs(diferenca) / y_test.values[0]) * 100
print(f"Erro percentual: {erro_percentual:.2f}%")


# --- 4. Salvando o CSV ---
print("\n--- Salvando o DataFrame de Faturamento Mensal ---")
monthly_revenue.to_csv('olist_monthly_revenue.csv', index=False)
print("Arquivo 'olist_monthly_revenue.csv' criado com sucesso!")

Recriando e limpando o DataFrame de faturamento mensal...


C:\Users\pacie\AppData\Local\Temp\ipykernel_28908\2723260769.py:38: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  pd.Grouper(key='order_purchase_datetime', freq='M')



--- Previsão de Faturamento do Último Mês ---
Dados de treino: 22 meses
Dados de teste: 1 mês

--- Análise da Predição ---
Faturamento real do último mês: R$ 589.67
Faturamento previsto para o último mês: R$ 549333.32
Diferença (Real - Previsto): R$ -548743.65
Erro percentual: 93059.45%

--- Salvando o DataFrame de Faturamento Mensal ---
Arquivo 'olist_monthly_revenue.csv' criado com sucesso!


- Os dados são insuficientes para gerar um treinamento que possa predizer com um grau elevado de confiança o faturamento do mês alvo.
- Confira o gráfico gerado no arquivo EDA onde mostra o aumento e diminuição das vendas ao decorrer dos meses.

### Churn Rate (evasão de um cliente)

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from datetime import datetime
import joblib
import os

# Supondo que df_orders, df_order_payments e df_orders_reviews já estão carregados na sua sessão.

# FUNÇÃO DE CONVERSÃO: Necessária para lidar com os timestamps
def safe_timestamp_to_datetime(series):
    series = pd.to_numeric(series, errors='coerce')
    if series.dropna().empty:
        return pd.NaT
    max_val = series.max()
    if max_val > 1e18:           # nanosegundos
        unit = 'ns'
    elif max_val > 1e12:         # milissegundos
        unit = 'ms'
    else:                        # segundos
        unit = 's'
    return pd.to_datetime(series, unit=unit, errors='coerce')


# --- 1. Preparação dos Dados para Análise RFM ---
print("Preparando dados para análise RFM e predição de Churn...")

# Converte datas para o tipo datetime usando a função segura
df_orders['order_purchase_datetime'] = safe_timestamp_to_datetime(df_orders['order_purchase_timestamp'])

# CORREÇÃO: Converte a coluna 'review_score' para numérico antes de ser usada no groupby
df_orders_reviews['review_score'] = pd.to_numeric(df_orders_reviews['review_score'], errors='coerce')

df_order_payments['payment_value'] = pd.to_numeric(df_order_payments['payment_value'], errors='coerce')


# Cria um DataFrame unificado para clientes e pedidos
df_customers_orders = df_orders.merge(
    df_order_payments[['order_id', 'payment_value']],
    on='order_id',
    how='inner'
)

# Adiciona a nota de avaliação à tabela de clientes
df_customers_reviews = df_orders.merge(
    df_orders_reviews[['order_id', 'review_score']],
    on='order_id',
    how='inner'
)


# --- 2. Criação das Features RFM (Recência, Frequência, Valor) ---

# Define a data de referência como a data da última compra do dataset + 1 dia
reference_date = df_customers_orders['order_purchase_datetime'].max() + pd.DateOffset(days=1)

# Calcula as métricas RFM
rfm_df = df_customers_orders.groupby('customer_id').agg(
    recency_days=('order_purchase_datetime', lambda x: (reference_date - x.max()).days),
    frequency_orders=('order_id', 'count'),
    monetary_value=('payment_value', 'sum')
).reset_index()

# Calcula a nota de avaliação média por cliente
avg_review_score = df_customers_reviews.groupby('customer_id')['review_score'].mean().reset_index()
rfm_df = rfm_df.merge(avg_review_score, on='customer_id', how='left')


# --- 3. Definição da Variável Alvo 'is_churn' ---
# Define churn como clientes que não compraram nos últimos 90 dias
churn_window_days = 90
rfm_df['is_churn'] = rfm_df['recency_days'].apply(lambda x: 1 if x > churn_window_days else 0)

print("Dados RFM e Churn criados:")
print(rfm_df.head())


# --- 4. Modelagem para prever 'is_churn' (Classificação) ---
print("\n--- Modelagem para prever a evasão de clientes (Churn) ---")

# CORREÇÃO: A feature 'recency_days' foi removida, pois ela causa vazamento de dados.
features = ['frequency_orders', 'monetary_value', 'review_score']
target = 'is_churn'

# Trata nulos na review_score (para clientes sem avaliação)
rfm_df['review_score'] = pd.to_numeric(rfm_df['review_score'], errors='coerce')
rfm_df['review_score'].fillna(rfm_df['review_score'].mean(), inplace=True)

X = rfm_df[features]
y = rfm_df[target]

# Padroniza as features numéricas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.25, random_state=42, stratify=y
)

models_classification = {
    "Logistic Regression": LogisticRegression(random_state=42),
    "Random Forest Classifier": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost Classifier": XGBClassifier(n_estimators=100, random_state=42)
}

rf_model_churn = None
for name, model in models_classification.items():
    print(f"\nTreinando {name}...")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print(f"Métricas para {name}:")
    print(f"Acurácia: {accuracy:.4f}")
    print(f"Precisão: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    
    if name == "Random Forest Classifier":
        rf_model_churn = model

# --- 💡 NOVO: Salvando o modelo e o scaler no novo diretório ---
print("\n--- Salvando o modelo e o scaler no novo diretório ---")
# 💡 CORREÇÃO: Usa o caminho '..' para subir um nível antes de entrar na pasta 'models'
model_dir = os.path.join('..', 'models', 'churn_rate_model')
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

if rf_model_churn is not None:
    joblib.dump(rf_model_churn, os.path.join(model_dir, 'rf_churn_classifier.joblib'))
    print("Modelo de churn salvo com sucesso!")
    
joblib.dump(scaler, os.path.join(model_dir, 'rfm_scaler.joblib'))
print("Scaler salvo com sucesso!")

Preparando dados para análise RFM e predição de Churn...
Dados RFM e Churn criados:
                        customer_id  recency_days  frequency_orders  \
0  00012a2ce6f8dcda20d059ce98491703           338                 1   
1  000161a058600d5901f007fab4c27140           459                 1   
2  0001fd6190edaaf884bcaf3d49edf079           597                 1   
3  0002414f95344307404f0ace7a26f1d5           428                 1   
4  000379cdec625522490c315e70c7a9fb           199                 1   

   monetary_value  review_score  is_churn  
0          114.74           1.0         1  
1           67.41           4.0         1  
2          195.42           5.0         1  
3          179.35           5.0         1  
4          107.01           4.0         1  

--- Modelagem para prever a evasão de clientes (Churn) ---

Treinando Logistic Regression...
Métricas para Logistic Regression:
Acurácia: 0.9042
Precisão: 0.9042
Recall: 1.0000
F1-Score: 0.9497

Treinando Random Forest Class

C:\Users\pacie\AppData\Local\Temp\ipykernel_28908\3029396872.py:92: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  rfm_df['review_score'].fillna(rfm_df['review_score'].mean(), inplace=True)


Métricas para Random Forest Classifier:
Acurácia: 0.8740
Precisão: 0.9213
Recall: 0.9411
F1-Score: 0.9311

Treinando XGBoost Classifier...
Métricas para XGBoost Classifier:
Acurácia: 0.9041
Precisão: 0.9042
Recall: 0.9999
F1-Score: 0.9497

--- Salvando o modelo e o scaler no novo diretório ---
Modelo de churn salvo com sucesso!
Scaler salvo com sucesso!


### Treinamento e salvando modelos Teste

In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import joblib
import os

# --- 1. Preparação dos Dados ---
print("Iniciando a preparação dos dados...")

# Lista completa de features numéricas e categóricas
all_numeric_features = [
    'price',
    'freight_value',
    'payment_installments',
    'total_delivery_time_hours',
    'shipping_time_hours',
    'product_weight_g',
    'product_volume_cm3'
]

categorical_features = [
    'customer_state'
]

# 💡 CORREÇÃO CRUCIAL: Converte todas as features numéricas para o tipo correto
for col in all_numeric_features:
    df_full_orders[col] = pd.to_numeric(df_full_orders[col], errors='coerce')

# Preenche os valores nulos com a mediana para as colunas numéricas
for col in all_numeric_features:
    median_val = df_full_orders[col].median()
    df_full_orders[col].fillna(median_val, inplace=True)

# 💡 CORREÇÃO: Garante que a coluna review_score seja numérica e sem nulos ANTES de usá-la
df_full_orders['review_score'] = pd.to_numeric(df_full_orders['review_score'], errors='coerce')
df_full_orders.dropna(subset=['review_score', 'delivery_delay_hours'], inplace=True)

# 💡 NOVO: Codifica as features categóricas com One-Hot Encoding
one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_features = one_hot_encoder.fit_transform(df_full_orders[categorical_features])
encoded_df = pd.DataFrame(encoded_features, columns=one_hot_encoder.get_feature_names_out(categorical_features))
encoded_df.index = df_full_orders.index

# Combina as features numéricas e categóricas
X_combined = pd.concat([df_full_orders[all_numeric_features], encoded_df], axis=1)


# --- 2. Modelagem para Prever 'is_satisfied' (Classificação) ---
print("\n--- Modelagem para prever se o cliente está satisfeito ---")

# 💡 CORREÇÃO: Garante que a coluna review_score seja numérica e sem nulos ANTES de usá-la
df_full_orders['is_satisfied'] = df_full_orders['review_score'].apply(lambda score: 1 if score >= 4 else 0)
X = X_combined
y_satisfied = df_full_orders['is_satisfied']

X_train, X_test, y_train, y_test = train_test_split(
    X, y_satisfied, test_size=0.20, random_state=42
)

models_classification = {
    "Logistic Regression": LogisticRegression(random_state=42, solver='liblinear'),
    "Random Forest Classifier": RandomForestClassifier(n_estimators=10, random_state=42),
    "XGBoost Classifier": XGBClassifier(n_estimators=10, random_state=42)
}

for name, model in models_classification.items():
    print(f"\nTreinando {name}...")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print(f"Métricas para {name}:")
    print(f"Acurácia: {accuracy:.4f}")
    print(f"Precisão: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")

# --- 💡 NOVO: Salvando o modelo e o encoder ---
print("\n--- Salvando o modelo e o encoder ---")
# Define o diretório para salvar
model_dir = 'models'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Salva o modelo de classificação que você quer (ex: Random Forest)
joblib.dump(models_classification['Random Forest Classifier'], os.path.join(model_dir, 'rf_classifier_satisfied.joblib'))
# Salva o encoder para garantir que a API use a mesma codificação
joblib.dump(one_hot_encoder, os.path.join(model_dir, 'one_hot_encoder.joblib'))
print("Modelo e encoder salvos com sucesso!")


# --- 3. Modelagem para Prever 'delivery_delay_hours' (Regressão) ---
print("\n--- Previsão de Delivery Delay Hours ---")

# Garantindo que a coluna de atraso está pronta para o modelo
df_full_orders['delivery_delay_hours'] = pd.to_numeric(df_full_orders['delivery_delay_hours'], errors='coerce')
df_full_orders.dropna(subset=['delivery_delay_hours'], inplace=True)

X_delay = X_combined
y_delay = df_full_orders['delivery_delay_hours']

X_train_delay, X_test_delay, y_train_delay, y_test_delay = train_test_split(
    X_delay, y_delay, test_size=0.20, random_state=42
)

models_regression = {
    "Linear Regression": LinearRegression(),
    "Random Forest Regressor": RandomForestRegressor(n_estimators=10, random_state=42),
    "XGBoost Regressor": XGBRegressor(n_estimators=10, random_state=42)
}

for name, model in models_regression.items():
    print(f"\nTreinando {name}...")
    model.fit(X_train_delay, y_train_delay)
    y_pred_delay = model.predict(X_test_delay)
    r2 = r2_score(y_test_delay, y_pred_delay)
    mae = mean_absolute_error(y_test_delay, y_pred_delay)
    rmse = np.sqrt(mean_squared_error(y_test_delay, y_pred_delay))
    print(f"Métricas para {name}:")
    print(f"R²: {r2:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")

Iniciando a preparação dos dados...


C:\Users\pacie\AppData\Local\Temp\ipykernel_26324\643058211.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_full_orders[col].fillna(median_val, inplace=True)
C:\Users\pacie\AppData\Local\Temp\ipykernel_26324\643058211.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For


--- Modelagem para prever se o cliente está satisfeito ---

Treinando Logistic Regression...
Métricas para Logistic Regression:
Acurácia: 0.7902
Precisão: 0.7913
Recall: 0.9880
F1-Score: 0.8788

Treinando Random Forest Classifier...
Métricas para Random Forest Classifier:
Acurácia: 0.9959
Precisão: 0.9967
Recall: 0.9980
F1-Score: 0.9974

Treinando XGBoost Classifier...
Métricas para XGBoost Classifier:
Acurácia: 0.8010
Precisão: 0.8037
Recall: 0.9811
F1-Score: 0.8836

--- Salvando o modelo e o encoder ---
Modelo e encoder salvos com sucesso!

--- Previsão de Delivery Delay Hours ---

Treinando Linear Regression...
Métricas para Linear Regression:
R²: 0.4912
MAE: 128.0181
RMSE: 174.9696

Treinando Random Forest Regressor...
Métricas para Random Forest Regressor:
R²: 0.9848
MAE: 12.7521
RMSE: 30.2541

Treinando XGBoost Regressor...
Métricas para XGBoost Regressor:
R²: 0.5353
MAE: 118.3635
RMSE: 167.2154
